In [1]:
import sys
sys.path.append("../..")

import time
from time import sleep
from datetime import datetime

# Import necessary libraries
import MetaTrader5 as mt5
import pandas as pd
from IPython.display import display
from lightweight_charts import JupyterChart
from IPython.display import clear_output, display

from sesto.fractal import high_low_finder
from sesto.utils import get_price_at_pnl, calculate_fee, calculate_position_size, get_pnl_at_price
from sesto.metatrader.business import get_positions, send_market_order, modify_sl_tp
from sesto.metatrader.data import fetch_data_pos
from sesto.constants import CURRENCIES, MT5Timeframe

MetaTrader5 initialized successfully
MetaTrader 5 initialized successfully.


In [2]:
# CURRENCIES = ['NZDJPY', 'AUDJPY', 'CADJPY', 'USDJPY']
CURRENCIES = ['BITCOIN', 'ETHEREUM', 'SOLANA']

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25
CAPITAL_PER_TRADE = 200  # $200 per trade
LEVERAGE = 500
DEVIATION = 10

TRAILING_STOP_STEPS = [
    {'trigger_tp_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.15},
    {'trigger_tp_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_tp_multiplier': 0.10, 'new_sl_pnl_multiplier': 0.01},
]

In [3]:
positions = get_positions()
positions

,ticket,time,time_msc,time_update,time_update_msc,type,magic,identifier,reason,volume,price_open,sl,tp,price_current,swap,profit,symbol,comment,external_id


In [4]:
MAIN_TIMEFRAME = MT5Timeframe.M15

In [7]:
size_usd = calculate_position_size(capital=CAPITAL_PER_TRADE, leverage=LEVERAGE)
fee = calculate_fee(position_size_usd=size_usd) 

while True:
    clear_output(wait=True)

    for currency in CURRENCIES:
        df = fetch_data_pos(currency, MAIN_TIMEFRAME, 5)
        df['fractal'] = high_low_finder(df)
        df['symbol'] = currency
        display(df.tail())

        last_row = df.iloc[-1]

        if df['fractal'].iloc[-1] == 'top':
            sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='short')
            tp = get_price_at_pnl(pnl_multiplier=TP_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='short')
            order = send_market_order(currency, 1, 'sell', sl, tp, DEVIATION)
            display(order)
        elif df['fractal'].iloc[-1] == 'bottom':
            sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='long')
            tp = get_price_at_pnl(pnl_multiplier=TP_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size_usd, leverage=LEVERAGE, entry_price=last_row['close'], type='long')
            order = send_market_order(currency, 1, 'buy', sl, tp, DEVIATION)
            display(order)

    positions = get_positions()
    
    for index, position in positions.iterrows():
        for trailing_step in TRAILING_STOP_STEPS:
            trigger_tp_multiplier, new_sl_pnl_multiplier = trailing_step['trigger_tp_multiplier'], trailing_step['new_sl_pnl_multiplier']
            pnl_threshold = TP_PNL_MULTIPLIER * trigger_tp_multiplier * CAPITAL_PER_TRADE

            if position.profit >= pnl_threshold:
                old_sl_price = position['sl']

                if position.type == 0:
                    new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=fee, position_size_usd=position.volume, leverage=LEVERAGE, entry_price=position.price_open, type='long')
                    if new_sl_price > old_sl_price:
                        modify_sl_tp(position.ticket, new_sl_price, position.tp)
                else:
                    new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=fee, position_size_usd=position.volume, leverage=LEVERAGE, entry_price=position.price_open, type='short')
                    if new_sl_price < old_sl_price:
                        modify_sl_tp(position.ticket, new_sl_price, position.tp)
                
                if new_sl_price:
                    capital_gain_at_current_unrealized_pnl = (position.profit / CAPITAL_PER_TRADE) * 100
                    old_sl_price_diff = (position.price_open / old_sl_price - 1) * 100
                    new_sl_price_diff = (position.price_open / position.sl - 1) * 100
                    pnl_at_old_sl = get_pnl_at_price(current_price=old_sl_price, entry_price=position.price_open, position_size_usd=size_usd, leverage=LEVERAGE, type='long' if position.type == 0 else 'short')
                    pnl_at_new_sl = get_pnl_at_price(current_price=position.sl, entry_price=position.price_open, position_size_usd=size_usd, leverage=LEVERAGE, type='long' if position.type == 0 else 'short')
                    capital_loss_at_old_sl = (pnl_at_old_sl / CAPITAL_PER_TRADE) * 100
                    capital_loss_at_new_sl= (pnl_at_new_sl / CAPITAL_PER_TRADE) * 100

                    print(f"{position.symbol} - TRAILING STOP - UNREALIZED PNL: ${position.profit:.3f} ({capital_gain_at_current_unrealized_pnl:.3f}% CAP) - OLD SL: ${old_sl_price:.3f} ({old_sl_price_diff:.3f}%)({capital_loss_at_old_sl:.3f}% CAP) - NEW SL: ${position.sl:.3f} ({new_sl_price_diff:.3f}%)({capital_loss_at_new_sl:.3f}% CAP)")
                    break  # Exit the loop after adjusting the stop loss
        

    sleep(60)

,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
0,2024-09-30 09:45:00,64437.84,64513.40,64358.52,64358.52,574,0,0,0,BITCOIN
1,2024-09-30 10:00:00,64356.02,64683.33,64354.96,64604.42,631,0,0,0,BITCOIN
2,2024-09-30 10:15:00,64605.26,64690.50,64540.95,64583.81,679,0,0,top,BITCOIN
3,2024-09-30 10:30:00,64584.47,64631.45,64531.46,64552.05,1005,0,0,0,BITCOIN
4,2024-09-30 10:45:00,64550.05,64606.89,64547.43,64570.96,433,0,0,0,BITCOIN


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
0,2024-09-30 09:45:00,2632.39,2635.07,2629.06,2629.74,1231,0,0,0,ETHEREUM
1,2024-09-30 10:00:00,2629.83,2643.48,2629.54,2639.28,1319,0,0,top,ETHEREUM
2,2024-09-30 10:15:00,2639.20,2641.98,2637.19,2640.63,949,0,0,0,ETHEREUM
3,2024-09-30 10:30:00,2640.50,2642.48,2635.05,2635.41,880,0,0,0,ETHEREUM
4,2024-09-30 10:45:00,2635.39,2638.94,2635.28,2638.14,412,0,0,0,ETHEREUM


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
0,2024-09-30 09:45:00,156.59,156.76,156.27,156.39,278,7,0,0,SOLANA
1,2024-09-30 10:00:00,156.38,157.34,156.17,157.21,404,9,0,0,SOLANA
2,2024-09-30 10:15:00,157.21,157.59,156.90,157.35,440,13,0,top,SOLANA
3,2024-09-30 10:30:00,157.35,157.61,156.60,156.67,388,7,0,top,SOLANA
4,2024-09-30 10:45:00,156.70,157.06,156.58,156.80,152,17,0,0,SOLANA


<class 'pandas.core.frame.DataFrame'>
